In [7]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import re
import datetime
import pandas as pd
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool
import warnings
from dotenv import load_dotenv

In [6]:
# === Env Keys ===
load_dotenv()
SERPER_API_KEY = os.environ['SERPER_API_KEY']
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

In [ ]:
warnings.filterwarnings("ignore")

# === Setup LLM + Search Tool ===
llm = LLM(model="gemini/gemini-2.0-flash")
search_tool = SerperDevTool(description="Search for BinID, PCN, GroupID and Plan Type")

# === Agents ===

research_agent = Agent(
    role="Plan Finder",
    goal="Identify the plan type and provide a reliable source PDF link for given BIN/PCN/GroupID.",
    backstory=(
        "You are a pharmacy plan expert who finds accurate and verifiable insurance plan info from payer websites, "
        "government portals, or PDF documentation. You must be confident in your result and verify the accuracy using multiple sources if needed."
    ),
    tools=[search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=False
)

verifier_agent = Agent(
    role="Plan Verifier",
    goal="Review and confirm that the plan type and PDF link provided are accurate and from official sources.",
    backstory="You're a detail-oriented verifier who double-checks facts and confirms reliability of insurance info.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

# === Input Entries ===
entries = [
    "610144~ABCCASHB~NULL",
    "NULL~NULL~WKLA",
    "004336~NULL~RX1123",
    "016820~NULL~TENNCARE",
    "006558~IBEW701~NULL",
    "021684~NULL~BXGI",
    "610014~NULL~PGE0000",
    "610342~HCICNA~NULL",
    "610014~NULL~GENMILS",
    "NULL~NULL~RX7770",
    "004336~NULL~RX1022",
    "610494~4848~NULL",
    "600428~03970000~NULL",
    "610014~NULL~TAMUSRX",
    "610014~NULL~STNDACTIVE",
    "004336~NULL~RX1054",
    "610014~NULL~1XS000000523643",
    "009893~RORIX~NULL",
    "004336~NULL~RX3956",
    "NULL~NULL~RX2408",
    "022188~PSTMEDC~NULL",
    "009513~NULL~NULL",
    "012353~06244200~NULL"
]

# === Helper to Parse Entry ===
def parse_entry(entry):
    parts = entry.split("~")
    return {
        "BIN": parts[0] if parts[0] != "NULL" else "",
        "PCN": parts[1] if parts[1] != "NULL" else "",
        "GROUP": parts[2] if parts[2] != "NULL" else ""
    }

results = []

# === Main Loop ===
for entry in entries:
    parsed = parse_entry(entry)

    # ---- Task 1: Research ----
    task1 = Task(
        description=f"""Search official sources to identify the **Plan Type** and **provide a valid source PDF link**
for this insurance info:

BIN: {parsed['BIN']}
PCN: {parsed['PCN']}
Group ID: {parsed['GROUP']}

Double-check all results and cite the most reliable source.

Output format (verbatim):
Plan Type: <type>
PDF Link: <url>
""",
        expected_output="Plan Type: <type>\nPDF Link: <link>",
        agent=research_agent
    )

    crew1 = Crew(
        agents=[research_agent],
        tasks=[task1],
        verbose=True,
        process=Process.sequential
    )

    result1 = crew1.kickoff()
    result_text = str(result1)

    # ---- Parse Research Results ----
    plan_type = re.search(r"Plan Type: (.*)", result_text)
    pdf_link = re.search(r"PDF Link: (.*)", result_text)

    original_plan = plan_type.group(1).strip() if plan_type else "Not Found"
    original_link = pdf_link.group(1).strip() if pdf_link else "Not Found"

    # ---- Task 2: Verification ----
    task2 = Task(
        description=f"""Verify the following information for BIN: {parsed['BIN']}, PCN: {parsed['PCN']}, Group ID: {parsed['GROUP']}:

Plan Type: {original_plan}
PDF Link: {original_link}

Check using authoritative sources (payer sites, CMS, or government PDFs).
If incorrect or incomplete, fix the value.

Expected format:
Verified Plan Type: <type>
Verified PDF Link: <url>
""",
        expected_output="Verified Plan Type: <type>\nVerified PDF Link: <url>",
        agent=verifier_agent
    )

    crew2 = Crew(
        agents=[verifier_agent],
        tasks=[task2],
        verbose=True,
        process=Process.sequential
    )

    result2 = crew2.kickoff()
    verify_text = str(result2)

    verified_plan = re.search(r"Verified Plan Type: (.*)", verify_text)
    verified_link = re.search(r"Verified PDF Link: (.*)", verify_text)

    verified_plan_value = verified_plan.group(1).strip() if verified_plan else original_plan
    verified_link_value = verified_link.group(1).strip() if verified_link else original_link

    # ---- Store Final Results ----
    results.append({
        "BIN": parsed['BIN'],
        "PCN": parsed['PCN'],
        "GroupID": parsed['GROUP'],
        "Original Plan Type": original_plan,
        "Original PDF Link": original_link,
        "Verified Plan Type": verified_plan_value,
        "Verified PDF Link": verified_link_value
    })

# === Export to Excel ===
df = pd.DataFrame(results)
filename = f"plan_results_verified_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df.to_excel(filename, index=False)

print(f"\n✅ Excel file saved as: {filename}")


2025-05-21 13:03:41,281 - 21112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Plan Finder
## Task: Search official sources to identify the **Plan Type** and **provide a valid source PDF link**
for this insurance info:

BIN: 610144
PCN: ABCCASHB
Group ID: 

Double-check all results and cite the most reliable source.

Output format (verbatim):
Plan Type: <type>
PDF Link: <url>



# Agent: Plan Finder
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"BIN 610144 PCN ABCCASHB\"}"
## Tool Output: 
{'searchParameters': {'q': 'BIN 610144 PCN ABCCASHB', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '[PDF] Pharmacy Point of Sale (ABSP) - Indian Health Service', 'link': 'https://www.ihs.gov/rpms/packagedocs/ABSP/absp0100.52o.pdf', 'snippet': '... BIN Number: 610144. Cardholder First Name: PATIENTAB. Cardholder ID: 123456789. Cardholder Last Name: POS. Claim ID: P20-610144-100011. Created On: MAY 19,2020 ...', 'position': 1}, {'title': '[PDF] ALLOCATION GUIDE - Good Neighbor Pharmacy', 'link': 'https://www.wearegnp.c

2025-05-21 13:04:15,720 - 21112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Plan Finder
## Final Answer: 
Plan Type: Cash Claim
PDF Link: https://www.ihs.gov/rpms/packagedocs/ABSP/absp0100.52o.pdf


# Agent: Plan Verifier
## Task: Verify the following information for BIN: 610144, PCN: ABCCASHB, Group ID: :

Plan Type: Cash Claim
PDF Link: https://www.ihs.gov/rpms/packagedocs/ABSP/absp0100.52o.pdf

Check using authoritative sources (payer sites, CMS, or government PDFs).
If incorrect or incomplete, fix the value.

Expected format:
Verified Plan Type: <type>
Verified PDF Link: <url>



2025-05-21 13:04:45,296 - 21112 - serper_dev_tool.py-serper_dev_tool:170 - ERROR: Error making request to Serper API: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


...



# Agent: Plan Verifier
## Final Answer: 
Verified Plan Type: Medicaid
Verified PDF Link: https://www.tnpharm.org/wp-content/uploads/12.2.19_TennCare_and_CoverRx_New_Plan_Implementations.pdf


# Agent: Plan Finder
## Task: Search official sources to identify the **Plan Type** and **provide a valid source PDF link**
for this insurance info:

BIN: 006558
PCN: IBEW701
Group ID: 

Double-check all results and cite the most reliable source.

Output format (verbatim):
Plan Type: <type>
PDF Link: <url>



# Agent: Plan Finder
## Thought: Okay, I need to find the Plan Type and a reliable PDF link for the insurance information provided: BIN 006558, PCN IBEW701, and Group ID (which is empty in this case). I will use the search tool to find this information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"BIN 006558 PCN IBEW701\"}"
## Tool Output: 
{'searchParameters': {'q': 'BIN 006558 PCN IBEW701', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title':

2025-05-21 13:08:13,323 - 21112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Plan Finder
## Final Answer: 
Plan Type: Pharmacy Benefit Manager (PBM) - often secondary coverage
PDF Link: http://www.4thdistricthealthfund.com/wp-content/uploads/2018/05/IBEW-4th_CARD.pdf


# Agent: Plan Verifier
## Task: Verify the following information for BIN: 006558, PCN: IBEW701, Group ID: :

Plan Type: Pharmacy Benefit Manager (PBM) - often secondary coverage
PDF Link: http://www.4thdistricthealthfund.com/wp-content/uploads/2018/05/IBEW-4th_CARD.pdf

Check using authoritative sources (payer sites, CMS, or government PDFs).
If incorrect or incomplete, fix the value.

Expected format:
Verified Plan Type: <type>
Verified PDF Link: <url>



# Agent: Plan Verifier
## Thought: I need to verify the plan type and PDF link provided for the given BIN, PCN, and Group ID. I will start by searching the internet for information related to these identifiers and the IBEW 4th District Health Fund to confirm the accuracy of the provided details.
## Using tool: Search the internet
## 

2025-05-21 13:08:36,763 - 21112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Plan Verifier
## Final Answer: 
Verified Plan Type: Pharmacy Benefit Manager (PBM) - often secondary coverage
Verified PDF Link: http://www.4thdistricthealthfund.com/wp-content/uploads/2018/05/IBEW-4th_CARD.pdf


# Agent: Plan Finder
## Task: Search official sources to identify the **Plan Type** and **provide a valid source PDF link**
for this insurance info:

BIN: 021684
PCN: 
Group ID: BXGI

Double-check all results and cite the most reliable source.

Output format (verbatim):
Plan Type: <type>
PDF Link: <url>

